In [9]:
'''
Task:	Scan the table to create an inverse kinematics dataset
Group:	19
Member:
	Antonio Sessa 0622702305 a.sessa108@studenti.unisa.it	
	Angelo Molinario 0622702311 a.molinario3@studenti.unisa.it
	Massimiliano Ranauro 0622702373 m.ranauro2@studenti.unisa.it
	Pietro Martano 0622702402 p.martano@studenti.unisa.it

'''

import numpy as np
import time
import csv
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter

df = pd.read_csv('ik_dataset_6joints.csv')

X = df[['TARGET_X', 'TARGET_Y', 'TARGET_Z']].values
y = df[['JOINT0', 'JOINT1', 'JOINT3', 'JOINT5', 'JOINT7', 'JOINT9']].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

class InverseKinematicsDataset(Dataset):
    def __init__(self, inputs, targets):
        self.inputs = torch.tensor(inputs, dtype=torch.float32)
        self.targets = torch.tensor(targets, dtype=torch.float32)

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return self.inputs[idx], self.targets[idx]

train_dataset = InverseKinematicsDataset(X_train, y_train)
test_dataset = InverseKinematicsDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [10]:
class InverseKinematicsModel(nn.Module):
    def __init__(self):
        super(InverseKinematicsModel, self).__init__()
        self.fc1 = nn.Linear(3, 256)
        self.fc2 = nn.Linear(256, 6)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)

        return x

In [12]:
device = torch.device('cpu')
model = InverseKinematicsModel().to(device)
loss_f = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)



# Training loop
num_epochs = 300

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_f(outputs, targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}")

# Evaluation
model.eval()
test_loss = 0.0
with torch.no_grad():
    for inputs, targets in test_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)
        loss = loss_f(outputs, targets)
        test_loss += loss.item() * inputs.size(0)

test_loss /= len(test_loader.dataset)
print(f"Test Loss: {test_loss:.4f}")
torch.save(model.state_dict(), 'ik_model.pth')

/usr/local/lib/python3.10/dist-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


KeyboardInterrupt: 